In [ ]:
#!pip install xgboost

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from transformers import RobertaTokenizer, RobertaForSequenceClassification, RobertaModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import re
from torch.optim import AdamW
import torch.nn as nn
from transformers import RobertaModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import xgboost as xgb

In [ ]:
# Load data
#tweet = pd.read_csv('labeled_data.csv', delimiter=',', engine='python', encoding='utf-8-sig')
tweet = pd.read_csv('HateSpeechDetection.csv', delimiter=',', engine='python', encoding='utf-8-sig')
#tweet = pd.read_csv('toxictweets(main dataset).csv', delimiter=',', engine='python', encoding='utf-8-sig')
#tweet.drop('id', axis=1, inplace=True)
#tweet.drop(['count', 'hate_speech', 'offensive_language', 'neither','Unnamed: 0'], axis=1, inplace=True)
tweet.drop('Platform', axis=1, inplace=True)
df = tweet.copy()

In [ ]:
df.head()

,Comment,Hateful
0,Damn I thought they had strict gun laws in Ger...,0
1,I dont care about what it stands for or anythi...,0
2,It's not a group it's an idea lol,0
3,So it's not just America!,0
4,The dog is a spectacular dancer considering he...,0


In [ ]:
# Preprocess text data
def preprocess(text):
    text = re.sub(r'@\w+', '', text)  # Remove @mentions
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'#', '', text)  # Remove hashtags
    return text

df['Comment'] = df['Comment'].apply(preprocess)
#df['tweet'] = df['tweet'].apply(preprocess)

In [ ]:
df.head(10)

,Comment,Hateful
0,Damn I thought they had strict gun laws in Ger...,0
1,I dont care about what it stands for or anythi...,0
2,It's not a group it's an idea lol,0
3,So it's not just America!,0
4,The dog is a spectacular dancer considering he...,0
5,If ppl dont wear masks you complain... If ppl ...,0
6,We should send them All back to africa,1
7,Checking to see if it's whataboutism from the ...,0
8,"As a european, I approve this.",0
9,"Idk which of these groups to join, they're all...",0


In [ ]:
#!pip install contractions
#!pip install emoji

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from contractions import fix
import emoji

# Download required resources for NLTK
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/uzairiftikhar515/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/uzairiftikhar515/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/uzairiftikhar515/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
# Initialize stop words and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Function to remove emojis
def remove_emoji(text):
    return emoji.replace_emoji(text, replace="")

# Function to preprocess text
def preprocess_text2(text):

    text = fix(text)

    # Remove mentions, URLs, and hashtags (already in your code, combining it here)
    text = re.sub(r'@\w+', '', text)  # Remove @mentions
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'#\w+', '', text)  # Remove hashtags

    # Convert text to lowercase
    text = text.lower()

    # Remove emojis
    text = remove_emoji(text)

    # Remove special characters, numbers, and punctuation
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Tokenize text by splitting on whitespace
    tokens = text.split()

    # Remove stop words and lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    # Join tokens back into a single string
    processed_text = ' '.join(tokens)

    return processed_text

In [ ]:
# Apply preprocessing to the dataset
#df['tweet'] = df['tweet'].apply(preprocess_text2)
df['Comment'] = df['Comment'].apply(preprocess_text2)

In [ ]:
df.head(10)

,Comment,Hateful
0,damn thought strict gun law germany,0
1,care stand anything connected like shield,0
2,group idea lol,0
3,america,0
4,dog spectacular dancer considering two left foot,0
5,people wear mask complain people wear mask als...,0
6,send back africa,1
7,checking see whataboutism past yup indeed what...,0
8,european approve,0
9,know group join cool,0


In [ ]:
class HateSpeechDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts.iloc[item])
        label = self.labels.iloc[item]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Load RoBERTa tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-large')
#roberta = RobertaModel.from_pretrained('roberta-large').to(device)

/home/uzairiftikhar515/Downloads/yes/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
class HateSpeechClassifier(nn.Module):
    def __init__(self, n_classes):
        super(HateSpeechClassifier, self).__init__()
        self.roberta = RobertaModel.from_pretrained('roberta-large')
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.roberta.config.hidden_size, n_classes)

    def forward(self, input_ids, attention_mask):
        _, pooled_output = self.roberta(
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=False
        )
        output = self.drop(pooled_output)
        return self.out(output)

model = HateSpeechClassifier(n_classes=2)
model = model.to(device)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Split data into training, validation, and test sets
#train_texts, test_texts, train_labels, test_labels = train_test_split(df['tweet'], df['class'], test_size=0.2, random_state=11)
train_texts, test_texts, train_labels, test_labels = train_test_split(df['Comment'], df['Hateful'], test_size=0.2, random_state=11)
#train_texts, test_texts, train_labels, test_labels = train_test_split(df['tweet'], df['Toxicity'], test_size=0.2, random_state=11)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.25, random_state=11)

In [ ]:
def collate_fn(batch):
    input_ids = torch.stack([x['input_ids'] for x in batch])
    attention_mask = torch.stack([x['attention_mask'] for x in batch])
    labels = torch.stack([x['labels'] for x in batch])
    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}

In [ ]:
# Define constants
MAX_LEN = 130
BATCH_SIZE = 18
EPOCHS = 5

In [ ]:
# Create DataLoader
train_dataset = HateSpeechDataset(train_texts, train_labels, tokenizer, MAX_LEN)
val_dataset = HateSpeechDataset(val_texts, val_labels, tokenizer, MAX_LEN)
test_dataset = HateSpeechDataset(test_texts, test_labels, tokenizer, MAX_LEN)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn)

In [ ]:
optimizer = AdamW(model.parameters(), lr=2e-5)
total_steps = len(train_dataloader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)
criterion = nn.CrossEntropyLoss().to(device)

In [ ]:
def train_epoch(model, dataloader, optimizer, criterion, scheduler, device):
    model.train()
    total_loss = 0
    correct_train_predictions = 0
    total_train_examples = 0

    for batch in dataloader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
        _, preds = torch.max(outputs, dim=1)
        correct_train_predictions += torch.sum(preds == labels)
        total_train_examples += labels.size(0)

    avg_train_loss = total_loss / len(dataloader)
    train_accuracy = correct_train_predictions.double() / total_train_examples

    return avg_train_loss, train_accuracy.item()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

def validate_epoch(model, dataloader, device):
    model.eval()
    correct_val_predictions = 0
    total_val_examples = 0
    val_labels_list = []
    val_preds_list = []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)

            val_labels_list.extend(labels.cpu().numpy())
            val_preds_list.extend(preds.cpu().numpy())

            correct_val_predictions += torch.sum(preds == labels)
            total_val_examples += labels.size(0)

    val_accuracy = correct_val_predictions.double() / total_val_examples
    print(confusion_matrix(val_labels_list, val_preds_list))
    print(classification_report(val_labels_list, val_preds_list, digits=4))

    return val_accuracy.item()

In [ ]:
training_accuracies = []
validation_accuracies = []

for epoch in range(EPOCHS):
    print(f'Epoch {epoch + 1}/{EPOCHS}')

    # Training for one epoch
    train_loss, train_accuracy = train_epoch(model, train_dataloader, optimizer, criterion, scheduler, device)
    training_accuracies.append(train_accuracy)

    print(f'Training loss for epoch {epoch + 1}: {train_loss}')
    print(f'Training Accuracy for epoch {epoch + 1}: {train_accuracy}')

    # Validation for one epoch
    val_accuracy = validate_epoch(model, val_dataloader, device)
    validation_accuracies.append(val_accuracy)

    print(f'Validation Accuracy for epoch {epoch + 1}: {val_accuracy}')

# Calculate average training and validation accuracies
avg_training_accuracy = sum(training_accuracies) / len(training_accuracies)
avg_validation_accuracy = sum(validation_accuracies) / len(validation_accuracies)

print(f'Average Training Accuracy: {avg_training_accuracy}')
print(f'Average Validation Accuracy: {avg_validation_accuracy}')

Epoch 1/5
Training loss for epoch 1: 0.3506214136630297
Training Accuracy for epoch 1: 0.8594444444444445
[[489   7]
 [ 23  81]]
              precision    recall  f1-score   support

           0     0.9551    0.9859    0.9702       496
           1     0.9205    0.7788    0.8438       104

    accuracy                         0.9500       600
   macro avg     0.9378    0.8824    0.9070       600
weighted avg     0.9491    0.9500    0.9483       600

Validation Accuracy for epoch 1: 0.9500000000000001
Epoch 2/5
Training loss for epoch 2: 0.12880987319163978
Training Accuracy for epoch 2: 0.9561111111111111
[[487   9]
 [ 18  86]]
              precision    recall  f1-score   support

           0     0.9644    0.9819    0.9730       496
           1     0.9053    0.8269    0.8643       104

    accuracy                         0.9550       600
   macro avg     0.9348    0.9044    0.9187       600
weighted avg     0.9541    0.9550    0.9542       600

Validation Accuracy for epoch 2: 0.

In [ ]:
def extract_embeddings(dataloader, model, device):
    model.eval()
    embeddings = []
    labels = []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            outputs = model.roberta(input_ids=input_ids, attention_mask=attention_mask)
            pooled_output = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
            embeddings.append(pooled_output)
            labels.append(batch['labels'].cpu().numpy())

    return np.vstack(embeddings), np.concatenate(labels)

In [ ]:
train_embeddings, train_labels = extract_embeddings(train_dataloader, model, device)
test_embeddings, test_labels = extract_embeddings(test_dataloader, model, device)

In [ ]:
# Train the XGBoost model using the embeddings
xgb_model = xgb.XGBClassifier(learning_rate=0.01, n_estimators=400, max_depth=4, eval_metric='mlogloss')
xgb_model.fit(train_embeddings, train_labels)

# Make predictions and evaluate the model
predictions = xgb_model.predict(test_embeddings)
accuracy = accuracy_score(test_labels, predictions)
report = classification_report(test_labels, predictions, digits=7)
confusion = confusion_matrix(test_labels, predictions)

# Display the evaluation metrics
print("Accuracy:", f"{accuracy:.7f}")
print("confusion Matrix:\n", confusion)
print("Classification Report:\n", report)

Accuracy: 0.9650000
confusion Matrix:
 [[460   7]
 [ 14 119]]
Classification Report:
               precision    recall  f1-score   support

           0  0.9704641 0.9850107 0.9776833       467
           1  0.9444444 0.8947368 0.9189189       133

    accuracy                      0.9650000       600
   macro avg  0.9574543 0.9398738 0.9483011       600
weighted avg  0.9646964 0.9650000 0.9646572       600



In [ ]:
import seaborn as sns
from sklearn import metrics
import matplotlib.pyplot as plt

In [ ]:
pred = xgb_model.predict(test_embeddings)
cnf_matrix = metrics.confusion_matrix(test_labels, pred)
p = sns.heatmap(pd.DataFrame(cnf_matrix), annot = True, cmap = "viridis", fmt = 'g')
plt.ylabel('Actual Label')
plt.xlabel('Predicted Label')

# Save the plot as an image file
plt.savefig('confusion_matrix_HSD.png')
plt.close()